<a href="https://colab.research.google.com/github/felicengie/from_coffeemachine_to_machinelearning/blob/main/location_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Load the dataset

In [ ]:
df = pd.read_csv('/content/locations.csv')

df.head()


,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,0,MKYhiSeXjahxh9O4Gqnimw,grace-street-new-york,Grace Street,https://s3-media4.fl.yelpcdn.com/bphoto/o9yM0Y...,False,https://www.yelp.com/biz/grace-street-new-york...,3383,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,...,40.747858,-73.986559,17 W 32nd St,NaN,NaN,New York,10001,US,NY,"['17 W 32nd St', 'New York, NY 10001']"
1,1,sSQ3dtC8nbfeT6srFiiIlA,stumptown-coffee-roasters-new-york,Stumptown Coffee Roasters,https://s3-media1.fl.yelpcdn.com/bphoto/qrd91f...,False,https://www.yelp.com/biz/stumptown-coffee-roas...,1444,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,...,40.745666,-73.988092,20 W 29th St,NaN,The Ace Hotel,New York,10001,US,NY,"['20 W 29th St', 'The Ace Hotel', 'New York, N..."
2,2,1aD88qNRb08LGBkl84GQTA,little-collins-new-york-3,Little Collins,https://s3-media2.fl.yelpcdn.com/bphoto/PavRDA...,False,https://www.yelp.com/biz/little-collins-new-yo...,1134,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,...,40.752330,-73.973530,708 3rd Ave,NaN,NaN,New York,10017,US,NY,"['708 3rd Ave', 'New York, NY 10017']"
3,3,N4Pss0mR4ygDOoSRpM2FQA,culture-espresso-new-york,Culture Espresso,https://s3-media4.fl.yelpcdn.com/bphoto/lsHjtN...,False,https://www.yelp.com/biz/culture-espresso-new-...,1123,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,...,40.752192,-73.985756,72 W 38th St,NaN,NaN,New York,10018,US,NY,"['72 W 38th St', 'New York, NY 10018']"
4,4,HjGjGy3JhhRFcR7MaQLrjA,frisson-espresso-new-york,Frisson Espresso,https://s3-media2.fl.yelpcdn.com/bphoto/QT3E2y...,False,https://www.yelp.com/biz/frisson-espresso-new-...,888,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,...,40.760935,-73.988899,326 W 47th St,NaN,NaN,New York,10036,US,NY,"['326 W 47th St', 'New York, NY 10036']"


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url',
       'review_count', 'categories', 'rating', 'transactions', 'price',
       'phone', 'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [ ]:
df.dtypes

Unnamed: 0                    int64
id                           object
alias                        object
name                         object
image_url                    object
is_closed                      bool
url                          object
review_count                  int64
categories                   object
rating                      float64
transactions                 object
price                        object
phone                       float64
display_phone                object
distance                    float64
coordinates.latitude        float64
coordinates.longitude       float64
location.address1            object
location.address2            object
location.address3            object
location.city                object
location.zip_code             int64
location.country             object
location.state               object
location.display_address     object
dtype: object

In [ ]:
unstr_text_colnames = ['is_closed', 'url', 'image_url', 'location.address1', 'location.address2', 'location.address3', 'distance', 'phone', 'display_phone', 'coordinates.latitude', 'coordinates.longitude', 'location.state', 'location.city', 'location.country', 'price']

In [ ]:
df.drop(columns=unstr_text_colnames, inplace=True)

In [ ]:
df.shape

(931, 10)

In [ ]:
url_colnames = [x for x in list(df.columns) if "url" in x]
url_colnames

[]

In [ ]:
df.drop(columns=url_colnames, inplace=True)

In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'alias', 'name', 'review_count', 'categories',
       'rating', 'transactions', 'location.zip_code',
       'location.display_address'],
      dtype='object')

### Load and handle 0 values in demographics dataset

In [ ]:
# # Load 'demographics.xlsx'
# demographics_df = pd.read_excel('/content/demographics.xlsx')

# # Replace all 0 values in each column with the mean of that column
# demographics_df.replace(0.0, np.nan, inplace=True)  # Replace 0 with NaN
# demographics_df.fillna(demographics_df['Best Population Estimate'].mean(), inplace=True)  # Replace NaN with the mean of each column

### Merge the two dataframes on zipcode

In [ ]:
# Load 'demographics.xlsx'
demographics_df = pd.read_excel('/content/demographics.xlsx')

# Merge the DataFrames based on the 'location.zip_code' and 'Geography' columns
merged_df = df.merge(demographics_df, left_on='location.zip_code', right_on='Geography', how='left')


merged_df.drop(columns=['Geography', 'Unnamed: 0'], inplace=True)

# Replace all 0 values in each column with NaN
merged_df.replace(0.0, np.nan, inplace=True)  # Replace 0 with NaN

merged_df.head()

,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate
0,MKYhiSeXjahxh9O4Gqnimw,grace-street-new-york,Grace Street,3383,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"['pickup', 'delivery']",10001,"['17 W 32nd St', 'New York, NY 10001']",NaN
1,sSQ3dtC8nbfeT6srFiiIlA,stumptown-coffee-roasters-new-york,Stumptown Coffee Roasters,1444,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,['delivery'],10001,"['20 W 29th St', 'The Ace Hotel', 'New York, N...",NaN
2,1aD88qNRb08LGBkl84GQTA,little-collins-new-york-3,Little Collins,1134,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10017,"['708 3rd Ave', 'New York, NY 10017']",15514.0
3,N4Pss0mR4ygDOoSRpM2FQA,culture-espresso-new-york,Culture Espresso,1123,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"['delivery', 'pickup']",10018,"['72 W 38th St', 'New York, NY 10018']",8063.0
4,HjGjGy3JhhRFcR7MaQLrjA,frisson-espresso-new-york,Frisson Espresso,888,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10036,"['326 W 47th St', 'New York, NY 10036']",30437.0


## Data Preprocessing

In [ ]:
nan_count = np.sum(merged_df.isnull())
nan_count

id                            0
alias                         0
name                          0
review_count                  0
categories                    0
rating                        0
transactions                  0
location.zip_code             0
location.display_address      0
Best Population Estimate    158
dtype: int64

In [ ]:
merged_df.dropna(inplace=True)
merged_df.dtypes

id                           object
alias                        object
name                         object
review_count                  int64
categories                   object
rating                      float64
transactions                 object
location.zip_code             int64
location.display_address     object
Best Population Estimate    float64
dtype: object

### Add a new feature that shows how many people visit the coffee shop relative to the neighbourhood size

In [ ]:
# Calculate the ratio and add it as a new column
merged_df['Reviews to Population Ratio'] = merged_df['review_count'] / merged_df['Best Population Estimate']

merged_df.head()

,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate,Reviews to Population Ratio
2,1aD88qNRb08LGBkl84GQTA,little-collins-new-york-3,Little Collins,1134,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10017,"['708 3rd Ave', 'New York, NY 10017']",15514.0,0.073095
3,N4Pss0mR4ygDOoSRpM2FQA,culture-espresso-new-york,Culture Espresso,1123,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"['delivery', 'pickup']",10018,"['72 W 38th St', 'New York, NY 10018']",8063.0,0.139278
4,HjGjGy3JhhRFcR7MaQLrjA,frisson-espresso-new-york,Frisson Espresso,888,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10036,"['326 W 47th St', 'New York, NY 10036']",30437.0,0.029175
5,ntkrxOSYT5VtTL4Z5jmmeA,ground-central-new-york,Ground Central,701,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"['pickup', 'delivery']",10022,"['155 E 52nd St', 'New York, NY 10022']",33303.0,0.021049
6,U3Ueh2bHtp3DLjrdc5w5fQ,remi-flower-and-coffee-new-york-2,Remi Flower & Coffee,625,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,[],10017,"['906 2nd Ave', 'New York, NY 10017']",15514.0,0.040286


Using feature importance to generate a new score columns (based on the zipcode and coordinates column) to see if that's a good location for a coffee shop.  For example, at one zipcode, there will be some coffee shops and from that area calculate the average rating of each coffee shops, the higher the rating then the higher the score will be in that column

Group by 'location.zip_code' and calculate the average rating for each zip code

In [ ]:
# average_ratings have the data of the average rating of coffee shops based on the same zip code
average_ratings = merged_df.groupby('location.zip_code')['rating'].mean()

# Create a dictionary that maps zip codes to their corresponding average ratings
zip_to_average_rating = average_ratings.to_dict()

# Map the average ratings to your dataset using the 'location.zip_code' column
merged_df['average rating'] = merged_df['location.zip_code'].map(zip_to_average_rating)
merged_df.head()

,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate,Reviews to Population Ratio,average rating
2,1aD88qNRb08LGBkl84GQTA,little-collins-new-york-3,Little Collins,1134,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10017,"['708 3rd Ave', 'New York, NY 10017']",15514.0,0.073095,4.044118
3,N4Pss0mR4ygDOoSRpM2FQA,culture-espresso-new-york,Culture Espresso,1123,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"['delivery', 'pickup']",10018,"['72 W 38th St', 'New York, NY 10018']",8063.0,0.139278,4.230769
4,HjGjGy3JhhRFcR7MaQLrjA,frisson-espresso-new-york,Frisson Espresso,888,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"['pickup', 'delivery']",10036,"['326 W 47th St', 'New York, NY 10036']",30437.0,0.029175,4.162791
5,ntkrxOSYT5VtTL4Z5jmmeA,ground-central-new-york,Ground Central,701,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"['pickup', 'delivery']",10022,"['155 E 52nd St', 'New York, NY 10022']",33303.0,0.021049,4.076271
6,U3Ueh2bHtp3DLjrdc5w5fQ,remi-flower-and-coffee-new-york-2,Remi Flower & Coffee,625,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,[],10017,"['906 2nd Ave', 'New York, NY 10017']",15514.0,0.040286,4.044118


### create new column that lists all names of the cafes in that zipcode

Group by 'location.zip_code' and concatenate the names of coffee shops with the same zip code

In [ ]:
# Group the data by 'location.zip_code' and aggregate the 'name' column into a list
grouped = merged_df.groupby('location.zip_code')['name'].agg(list).reset_index()

# Merge the aggregated data back into the original DataFrame based on the 'location.zip_code'
merged_df = merged_df.merge(grouped, on='location.zip_code', suffixes=('', '_in_zip'))

# Rename the newly created column to 'names_in_zip'
merged_df.rename(columns={'name_in_zip': 'all_names'}, inplace=True)

merged_df['location.zip_code'].nunique()

59

In [ ]:
merged_df.shape

(773, 13)

### Sort df by highest to lowest average_rating

In [ ]:
sorted_df = merged_df.sort_values(by='review_count', ascending=False)
sorted_df.head()

,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate,Reviews to Population Ratio,average rating,all_names
354,H4jJ7XB3CetIr1pg56CczQ,levain-bakery-new-york-new-york-2,Levain Bakery - New York,9649,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"['pickup', 'delivery']",10023,"['167 W 74th St', 'New York, NY 10023']",65891.0,0.146439,3.940000,"[Black Press Coffee, Solid State Coffee, Commo..."
395,a0IET3_yCFcO36OqGSsisg,eataly-nyc-flatiron-new-york-3,Eataly NYC Flatiron,6102,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"['delivery', 'pickup']",10010,"['200 5th Ave', 'New York, NY 10010']",33193.0,0.183834,3.931818,"[Black Star Bakery & Cafe, Maman, Citizens Of ..."
113,U5hCNNyJmb7f3dmC1HTzSQ,juniors-restaurant-new-york-9,Junior's Restaurant,5882,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,"['pickup', 'delivery']",10036,"['1515 Broadway', 'New York, NY 10036']",30437.0,0.193252,4.162791,"[Frisson Espresso, Bird & Branch, For Five Cof..."
512,ga6sRtE0l85iftw_5-W84Q,dominique-ansel-bakery-new-york,Dominique Ansel Bakery,5391,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,['delivery'],10012,"['189 Spring St', 'New York, NY 10012']",22269.0,0.242085,4.107143,"[% Arabica, Blank Street Coffee, Café Lyria, C..."
151,c3eMI4_o4dPDDhPV_ibBYQ,ess-a-bagel-new-york,Ess-a-Bagel,4439,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.0,"['pickup', 'delivery']",10022,"['831 3rd Ave', 'New York, NY 10022']",33303.0,0.133291,4.076271,"[Ground Central, Slate Cafe Midtown, Gregorys ..."


### Add label column which is a binary classification (either optimal or nonoptimal) based on our criteria for each row

In [ ]:
median_ratio = sorted_df['Reviews to Population Ratio'].median()
median_avg_rating = sorted_df['average rating'].median()
# median_review_count = sorted_df['review_count'].median()

print('median_ratio:', median_ratio)
print('median_avg_rating:', median_avg_rating)
# print('median_review_count:', median_review_count)

median_ratio: 0.0017504376094023505
median_avg_rating: 4.166666666666667


In [ ]:
# Define your criteria for an optimal location
criteria = (
    (sorted_df['average rating'] >= median_avg_rating) &  # Example criteria: Good average rating
    (sorted_df['Reviews to Population Ratio'] >= median_ratio)  # Example criteria: High reviews to population ratio
)

# Create the 'optimal_location' column based on the criteria
sorted_df['optimal_location'] = 'No'  # Initialize all locations as 'No'
sorted_df.loc[criteria, 'optimal_location'] = 'Yes'  # Update locations that meet the criteria to 'Yes'

sorted_df.head()


,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate,Reviews to Population Ratio,average rating,all_names,optimal_location
354,H4jJ7XB3CetIr1pg56CczQ,levain-bakery-new-york-new-york-2,Levain Bakery - New York,9649,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"['pickup', 'delivery']",10023,"['167 W 74th St', 'New York, NY 10023']",65891.0,0.146439,3.940000,"[Black Press Coffee, Solid State Coffee, Commo...",No
395,a0IET3_yCFcO36OqGSsisg,eataly-nyc-flatiron-new-york-3,Eataly NYC Flatiron,6102,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"['delivery', 'pickup']",10010,"['200 5th Ave', 'New York, NY 10010']",33193.0,0.183834,3.931818,"[Black Star Bakery & Cafe, Maman, Citizens Of ...",No
113,U5hCNNyJmb7f3dmC1HTzSQ,juniors-restaurant-new-york-9,Junior's Restaurant,5882,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,"['pickup', 'delivery']",10036,"['1515 Broadway', 'New York, NY 10036']",30437.0,0.193252,4.162791,"[Frisson Espresso, Bird & Branch, For Five Cof...",No
512,ga6sRtE0l85iftw_5-W84Q,dominique-ansel-bakery-new-york,Dominique Ansel Bakery,5391,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,['delivery'],10012,"['189 Spring St', 'New York, NY 10012']",22269.0,0.242085,4.107143,"[% Arabica, Blank Street Coffee, Café Lyria, C...",No
151,c3eMI4_o4dPDDhPV_ibBYQ,ess-a-bagel-new-york,Ess-a-Bagel,4439,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.0,"['pickup', 'delivery']",10022,"['831 3rd Ave', 'New York, NY 10022']",33303.0,0.133291,4.076271,"[Ground Central, Slate Cafe Midtown, Gregorys ...",No


In [ ]:
print('Optimal: ', np.sum(sorted_df['optimal_location'] == 'Yes'))
print('Not Optimal: ', np.sum(sorted_df['optimal_location'] == 'No'))

Optimal:  203
Not Optimal:  570


#### Round values in 'Reviews to Population Ratio' to 3 decimal places

In [ ]:
sorted_df['Reviews to Population Ratio'] = sorted_df['Reviews to Population Ratio'].round(3)
sorted_df.head()

temp = sorted_df.sort_values(by='Reviews to Population Ratio', ascending=True)
temp.head()

,id,alias,name,review_count,categories,rating,transactions,location.zip_code,location.display_address,Best Population Estimate,Reviews to Population Ratio,average rating,all_names,optimal_location
143,iG_FcwMVKdtV2g7TNIjHhw,787-coffee-new-york-16,787 Coffee,1,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,['pickup'],10022,"['115 E 57th St', 'New York, NY 10022']",33303.0,0.0,4.076271,"[Ground Central, Slate Cafe Midtown, Gregorys ...",No
193,aQXWVZK5w0L3klxf2jpAaA,le-cafe-coffee-new-york-12,Le Cafe Coffee,11,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.5,"['delivery', 'pickup']",10019,"['629 W 57th St', 'New York, NY 10019']",43989.0,0.0,3.977273,"[Zibetto Espresso Bar, Ground Central, Birch C...",No
618,Wh05sPEf1rmgJfl7OHNlCw,felix-roasting-new-york-6,Felix Roasting,11,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,[],10013,"['525 Greenwich St', 'New York, NY 10013']",29563.0,0.0,4.296296,"[Koré Coffee, Now or Never Coffee, Maman, Inte...",No
522,e1GsbWEEDoxZywrYT-S7Ng,plantshed-new-york-6,Plantshed,11,"[{'alias': 'gardening', 'title': 'Nurseries & ...",4.5,[],10012,"['216 Lafayette St', 'New York, NY 10012']",22269.0,0.0,4.107143,"[% Arabica, Blank Street Coffee, Café Lyria, C...",No
428,_Wnr-QxHgtAlHOLgpnXLQQ,murphy-s-door-new-york,Murphy’s Door,11,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"['delivery', 'pickup']",10009,"['440 E 9th St', 'New York, NY 10009']",60000.0,0.0,4.363636,"[Le Phin, Hi-Note, Book Club Bar, A’MORE Caffe...",No


## Train a Random Forest Classifier model and test it with a test data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the feature columns
features = ['review_count', 'Best Population Estimate', 'Reviews to Population Ratio', 'average rating']

# Define the target variable
target = 'optimal_location'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sorted_df[features], sorted_df[target], test_size=0.2, random_state=42)

# Create and train a Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", report)

# Predict a new location's class (replace with real data)
new_location_data = pd.DataFrame({
    'review_count': [5391],
    'Best Population Estimate': [22269.0],
    'Reviews to Population Ratio': [0.242],
    'average rating': [4.107143]
})

predicted_class = classifier.predict(new_location_data)
print(f"Predicted Class for New Location: {predicted_class[0]}")

Model Accuracy: 0.99

Classification Report:
               precision    recall  f1-score   support

          No       0.99      0.99      0.99       114
         Yes       0.98      0.98      0.98        41

    accuracy                           0.99       155
   macro avg       0.98      0.98      0.98       155
weighted avg       0.99      0.99      0.99       155

Predicted Class for New Location: No
